NOMBRE: Jose Miguel Gonzalez


CARNE: 20335



FECHA: 10/06/2024

**Objetivo:** La idea de esta práctica es que puedan llevar a cabo una tarea de Name Entity Recognition (NER) con Bert y spantBert, y sean capaces de comparar cuál rinde mejor.

Responda las siguiente preguntas:
1. ¿Qué es una tarea ner?
2. ¿Cuál metrica de rendimiento debe usar en este caso?

In [1]:
#Librerías
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import numpy as np
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'datasets'

In [ ]:
#Dataset
dataset = load_dataset("conll2003")

In [ ]:
spanbert = "SpanBERT/spanbert-base-cased"

#Tokenizador
tokenizer = AutoTokenizer.from_pretrained(spanbert)

#Función de codificación
def tokenizeAndAlignLabels(examples):
    tokenizedInputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        is_split_into_words=True,
        padding="max_length",
        max_length=43
    )
    
    labels = []

    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenizedInputs.word_ids(i)  # Get the word ids for the tokens
        label_ids = [-100 if id is None else label[id] for id in word_ids]  # Align labels
        labels.append(label_ids)

    tokenizedInputs['labels'] = labels
    return tokenizedInputs

#Dataset codificado
tokenizedDataset = dataset.map(tokenizeAndAlignLabels, batched=True)

#Modelo
model = AutoModelForTokenClassification.from_pretrained(
    spanbert, 
    num_labels=dataset['train'].features['ner_tags'].feature.num_classes
)

#Métrica
metric = load("seqeval")

#Función de entrenamiento
def computeMetrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Eliminar las predicciones de los tokens de relleno
    truePredictions = [
        [dataset["train"].features["ner_tags"].feature.names[prediction] for idx, prediction in enumerate(examplePredictions) if labels[idx] != -100]
        for examplePredictions, labels in zip(predictions, labels)
    ]
    trueLabels = [
        [dataset["train"].features["ner_tags"].feature.names[label] for idx, label in enumerate(exampleLabels) if label != -100]
        for exampleLabels in labels
    ]

    results = metric.compute(predictions=truePredictions, references=trueLabels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

#Argumentos de entrenamiento
trainingArguments = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
)

#Entrenador
trainer = Trainer(
    model=model,
    args=trainingArguments,
    train_dataset=tokenizedDataset["train"],
    eval_dataset=tokenizedDataset["validation"],
    compute_metrics=computeMetrics
)

#Entrenamiento
trainer.train()

#Evaluación
evaluationSpanbert = trainer.evaluate()

In [ ]:
bert = "bert-base-uncased"

#Tokenizador
tokenizer = AutoTokenizer.from_pretrained(spanbert)

#Función de codificación
def tokenizeAndAlignLabels(examples):
    tokenizedInputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        is_split_into_words=True,
        padding="max_length",
        max_length=43
    )
    
    labels = []

    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenizedInputs.word_ids(i)  # Get the word ids for the tokens
        label_ids = [-100 if id is None else label[id] for id in word_ids]  # Align labels
        labels.append(label_ids)

    tokenizedInputs['labels'] = labels
    return tokenizedInputs

#Dataset codificado
tokenizedDataset = dataset.map(tokenizeAndAlignLabels, batched=True)

#Modelo
model = AutoModelForTokenClassification.from_pretrained(
    spanbert, 
    num_labels=dataset['train'].features['ner_tags'].feature.num_classes
)

#Métrica
metric = load("seqeval")

#Función de entrenamiento
def computeMetrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Eliminar las predicciones de los tokens de relleno
    truePredictions = [
        [dataset["train"].features["ner_tags"].feature.names[prediction] for idx, prediction in enumerate(examplePredictions) if labels[idx] != -100]
        for examplePredictions, labels in zip(predictions, labels)
    ]
    trueLabels = [
        [dataset["train"].features["ner_tags"].feature.names[label] for idx, label in enumerate(exampleLabels) if label != -100]
        for exampleLabels in labels
    ]

    results = metric.compute(predictions=truePredictions, references=trueLabels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

#Argumentos de entrenamiento
trainingArguments = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
)

#Entrenador
trainer = Trainer(
    model=model,
    args=trainingArguments,
    train_dataset=tokenizedDataset["train"],
    eval_dataset=tokenizedDataset["validation"],
    compute_metrics=computeMetrics
)

#Entrenamiento
trainer.train()

#Evaluación
evaluationBert = trainer.evaluate()

In [ ]:
# Compare the two models
print("SpanBERT:")
print(evaluationSpanbert)
print("BERT:")
print(evaluationBert)